In [2]:
from bs4 import BeautifulSoup
from rich import print
import httpx

In [31]:
def page_to_simple_dict(html_str: str) -> dict:
    """
    format like
    '作者': 'ReiKuromiya (ReiKuromiya)',
    '標題': '[正妹] 周子瑜',
    '時間': 'Sun Jan  1 00:26:06 2023',
    'Year': '2023',
    'Month': 'Jan',
    'Date': '',
    'Week': 'Sun',
    'Time': '00:26:06',
    'Body': [(..., ...),  ...]}
    """

    def to_detail_date(date_str: str) -> dict:
        detail_date = date_str.split(" ")
        return {
            "Year": detail_date[-1],
            "Month": detail_date[1],
            "Date": detail_date[2],
            "Week": detail_date[0],
            "Time": detail_date[-2],
        }

    # header = {"cookie": "over18=1"}
    # result = httpx.get(url=url, headers=header)

    soup = BeautifulSoup(html_str, "html.parser")

    # get main data
    body_data = soup.find("div", class_="bbs-screen bbs-content", id="main-content")

    # print()
    
    # get header data
    header_data = body_data.find_all("div", class_="article-metaline")

    tab_list = [
        str(line.find("span", class_="article-meta-tag").string) for line in header_data
    ]

    value_list = [
        str(line.find("span", class_="article-meta-value").string)
        for line in header_data
    ]

    header_dict = dict(zip(tab_list, value_list))
    print(header_dict)
    
    if "時間" not in header_dict:
        process_text = body_data.contents[2]
        process_text = process_text.split("\n")[:2]
        process_text = [item.split(":") for item in process_text]
        process_result_dict  = {item[0] : item[1] for item in process_text}
        header_dict |= process_result_dict
    
    
    
    # image src
    images_list = body_data.find_all("div", class_="richcontent")
    
    # base in how long in image_list
    images_list = [item.get("src") for image in images_list if (item := image.find("img"))]
    # print(images_list)

    # image link lists
    link_image = body_data.find_all("a")
    # {".png" , ".jpg" , "jpeg" ,".gif"} in (link_str := str(link.string))
    link_image = [link_str for link in link_image if any(substring in (link_str := str(link.string)) for substring in [".png" , ".jpg" , "jpeg" ,".gif"])  ]


    # content_images = list(zip(link_image[:len(images_list)], images_list))

    page_data = (
        header_dict | to_detail_date(header_dict["時間"]) | {"image_catch_list": images_list , "image_link" :link_image }
    )

    return page_data




In [22]:
result = httpx.get("https://www.ptt.cc/bbs/Beauty/M.1690589266.A.166.html" , headers={"cookie": "over18=1"})



In [23]:
result.text

'<!DOCTYPE html>\n<html>\n\t<head>\n\t\t<meta charset="utf-8">\n\t\t\n\n<meta name="viewport" content="width=device-width, initial-scale=1">\n\n<title>閱讀文章 - 看板 Beauty - 批踢踢實業坊</title>\n<meta name="robots" content="all">\n<meta name="keywords" content="Ptt BBS 批踢踢">\n<meta name="description" content="標題:[神人] 外國婚紗model\n時間: Sat Jul 29 08:07:44 2023\n大家星期六早安，\n最近在ig看到一個有仙氣的婚紗model，\n但完全找不到是誰QQ，\n">\n<meta property="og:site_name" content="Ptt 批踢踢實業坊">\n\n<meta property="og:description" content="標題:[神人] 外國婚紗model\n時間: Sat Jul 29 08:07:44 2023\n大家星期六早安，\n最近在ig看到一個有仙氣的婚紗model，\n但完全找不到是誰QQ，\n">\n<link rel="canonical" href="https://www.ptt.cc/bbs/Beauty/M.1690589266.A.166.html">\n\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-common.css">\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen">\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-custom.css">\n<link rel="stylesheet" type="text/css

In [32]:
test = page_to_simple_dict(result.text)

print(test)

{'作者': 'a2396494 (砒霜)'}

{
    '作者': 'a2396494 (砒霜)',
    '標題': '[神人] 外國婚紗model',
    '時間': ' Sat Jul 29 08',
    'Year': '08',
    'Month': 'Sat',
    'Date': 'Jul',
    'Week': '',
    'Time': '29',
    'image_catch_list': [
        'https://cache.ptt.cc/c/https/i.imgur.com/xKxiDzKl.jpg?e=1709222097&s=WGJAbrpLAB94U_SvtnRKLQ',
        'https://cache.ptt.cc/c/https/i.imgur.com/lFnxOql.png?e=1709215982&s=c_9qQ_fr-w8VpHqfG0ZfOg',
        'https://cache.ptt.cc/c/https/i.imgur.com/VR12YnZl.png?e=1709234214&s=pgj1BkiuPHz8dhWYPyBHCQ',
        'https://cache.ptt.cc/c/https/i.imgur.com/ysqiRmkl.jpg?e=1709194292&s=VuQsP6AaoxpsoSt2Z8-itA',
        'https://cache.ptt.cc/c/https/i.imgur.com/QVX5hQrl.jpg?e=1709230559&s=8lVfeIChVH80xv-ICEEL9A',
        'https://cache.ptt.cc/c/https/i.imgur.com/BEqRDYrl.jpg?e=1709202973&s=gTlh3RMoQgzVuWH12y0U7A'
    ],
    'image_link': [
        'https://i.imgur.com/xKxiDzK.jpg',
        'https://i.imgur.com/lFnxOqh.png',
        'https://i.imgur.com/VR12YnZ.png',
        'https://i.imgur.com/ysqiRmk.jpg',
        'https://i.imgur.com/QVX5hQr.jpg',
        'https://i.imgur.com/BEqRDYr.jpg'
    ]
}

In [79]:
string = "Sun Jan  1 20:04:22 2023"
detail_date = string.split(" ")
detail_date.remove("")
detail_date

['Sun', 'Jan', '1', '20:04:22', '2023']

In [80]:
{
    "Year": detail_date[-1],
    "Month": detail_date[1],
    "Date": detail_date[2],
    "Week": detail_date[0],
    "Time": detail_date[-2],
}

{'Year': '2023',
 'Month': 'Jan',
 'Date': '1',
 'Week': 'Sun',
 'Time': '20:04:22'}